<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/RAG/advanced_rag/hyde_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# HyDE


## Google Colab preparation[optional]
This is an optional step, if you want to run this notebook on Google Colab.

In [ ]:
! git clone https://github.com/milvus-io/bootcamp.git

In [ ]:
import shutil
src_dir = "./bootcamp/bootcamp/RAG/advanced_rag/rag_utils"
dst_dir = "./rag_utils"
shutil.copytree(src_dir, dst_dir)

In [ ]:
! pip install --upgrade langchain langchain-community langchain_milvus langchain-openai bs4

Please prepare you [OPENAI_API_KEY](https://openai.com/index/openai-api/) in your environment variables.
![](imgs/colab_api_key1.png)

### If you are running this notebook on Google Colab, you have to restart this session by `Cmd/Ctrl + M`, then press `.` to make the environment take effect.

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

----
## Get started
![](imgs/hyde.png)

## Prepare the data

We use the Langchain WebBaseLoader to load documents from [blog sources](https://lilianweng.github.io/posts/2023-06-23-agent/) and split them into chunks using the RecursiveCharacterTextSplitter.

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from rag_utils.vanilla import vectorstore

# Create a WebBaseLoader instance to load documents from web sources
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# Load documents from web sources using the loader
documents = loader.load()
# Initialize a RecursiveCharacterTextSplitter for splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Split the documents into chunks using the text_splitter
docs = text_splitter.split_documents(documents)

## Build the chain

We load the docs into milvus vectorstore, and build a milvus retriever.

In [2]:
vectorstore.add_documents(docs)
retriever = vectorstore.as_retriever()

Build the vanilla RAG chain.

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from rag_utils.vanilla import format_docs, rag_prompt, llm

vanilla_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

Build a hyde chain.

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from rag_utils.hyde import HydeRetriever

hyde_retriever = HydeRetriever.from_vectorstore(vectorstore)

hyde_chain = (
    {"context": hyde_retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

## Test the chain


In [5]:
query = "which vector approximate searching algorithms work in a vector store"

vanilla_result = vanilla_rag_chain.invoke(query)
hyde_result = hyde_chain.invoke(query)
print(f"\n[vanilla_result]:\n{vanilla_result}\n\n[hyde_result]:\n{hyde_result}")


[vanilla_result]:
The approximate nearest neighbors (ANN) algorithm is commonly used for vector approximate searching in a vector store. Some specific implementations of ANN algorithms include FAISS (Facebook AI Similarity Search), ScaNN (Scalable Nearest Neighbors), LSH (Locality-Sensitive Hashing), and ANNOY (Approximate Nearest Neighbors Oh Yeah).

[hyde_result]:
The vector approximate searching algorithms that work in a vector store include Locality-Sensitive Hashing (LSH), ANNOY (Approximate Nearest Neighbors Oh Yeah), FAISS (Facebook AI Similarity Search), ScaNN (Scalable Nearest Neighbors), and HNSW (Hierarchical Navigable Small World). These algorithms are commonly used for fast Maximum Inner Product Search (MIPS) in a vector store database.


In the [hyde_result], it retrieved with ground truth "HNSW", which does not appear in the [vanilla_result].

Let's dive deep into the retrieved results to find the reason.

In [8]:
# retriever.invoke(query)

![](imgs/hyde_res2.png)

In [9]:
# hyde_retriever.invoke(query)

![](imgs/hyde_res1.png)

The hyde_retriever retrieved the document with `HNSW`, while the vanilla_retriever did not. This is because HyDE generates fake documents that may contain various vector approximate searching algorithms like `HNSW`, which makes the retriever's results more accurate.